In [ ]:
!pip install contractions


In [ ]:
import pandas as pd
import numpy as np
import re
import nltk.corpus
nltk.download('stopwords')
nltk.download('words')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('all')
from nltk.corpus import stopwords, words
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import contractions
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale, MinMaxScaler
from skmultilearn.problem_transform import BinaryRelevance, LabelPowerset, ClassifierChain
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
test_label = pd.read_csv("test_labels.csv")
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

In [ ]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
# Merging test and train to form one huge dataset
test_data = pd.merge(test, test_label)
dataset = pd.concat([test_data, train])
dataset.drop(columns=['id'], inplace=True)
dataset.drop_duplicates(inplace=True, ignore_index=True)
dataset.drop(dataset.index[dataset['toxic'] == -1], inplace = True)
dataset.reset_index(inplace = True)
dataset

,index,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,5,Thank you for understanding. I think very high...,0,0,0,0,0,0
1,7,:Dear god this site is horrible.,0,0,0,0,0,0
2,11,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0
3,13,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0
4,14,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
223544,312730,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
223545,312731,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
223546,312732,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
223547,312733,And it looks like it was actually you who put ...,0,0,0,0,0,0


# Split dataset

In [ ]:
# Splitting into train test sets
X = dataset.drop(columns=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])
y = dataset[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].copy()

X_train, X_test_and_val, y_train, y_test_and_val = train_test_split(X,y, train_size=0.8)
X_val, X_test, y_val, y_test = train_test_split(X_test_and_val,y_test_and_val, train_size=0.5)

# TF-IDF

In [ ]:

vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')
vectorizer.fit(X_train['comment_text_cleaned'])
vectorizer.fit(X_val['comment_text_cleaned'])

TfidfVectorizer(ngram_range=(1, 3), strip_accents='unicode')

In [ ]:
x_train = vectorizer.transform(X_train['comment_text_cleaned'])
x_val = vectorizer.transform(X_val['comment_text_cleaned'])


In [ ]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


In [ ]:
#forest = RandomForestClassifier(n_estimators=1000, max_leaf_nodes=18, random_state=21)
#clf = MultiOutputClassifier(forest)
#clf.fit(x_train, y_train)
#pred = clf.predict(x_val) 

In [ ]:
preds = np.zeros((len(y_val), len(label_cols))) # empty np matrix to put in predictions

for i, j in enumerate(label_cols):
    print('fit', j)
    m = RandomForestClassifier(n_estimators=1000, max_leaf_nodes=18, random_state=21)
    m.fit(x_train, y_train[j].values)
    preds[:,i] = m.predict(x_val)




fit toxic
fit severe_toxic
fit obscene
fit threat
fit insult
fit identity_hate


In [ ]:
def evaluation_metric(y_true, y_pred):
  print(precision_score(y_true, y_pred, average='micro'))
  if(precision_score(y_true, y_pred, average='micro') > 0.1):
    print(recall_score(y_true, y_pred, average='micro'))
  else:
    print('Precision too low')

In [ ]:
preds

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [ ]:
print("Score = ",evaluation_metric(y_val, preds))

0.0
Precision too low
Score =  None


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=64e8a870-3b5a-497f-8555-18bf59239164' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>